In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd

from newrow import newrow_cash, newrow_etf
from utils import broker_fee, get_date
from fetch_data import fetch_ter

pd.set_option('display.max_columns', None)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Importa il report:
- Per iniziare da zero con un report nuovo, importare `report-template.csv`
- Altrimenti importare `report.csv`
- Una volta svolte le operazioni, per salvare le modifiche seguire le istruzioni in fondo al notebook

In [5]:
df = pd.read_csv("reports/report.csv")  # Da zero: reports/report-template.csv

## Visualizza il report:

In [ ]:
df

### *Rimuovi manualmente l'ultima riga:*
- La cella qui sotto rimuove l'ultima riga del report. E' un "tasto elimina" manuale. 
- Ad esempio runnando la cella 3 volte rimuovo le ultime 3 righe

In [8]:
df = df.iloc[:-1]

## Programma principale
- Runnare la cella sottostante per avviare il programma e seguire le istruzioni a schermo

### IMPLEMENTATI:

1. ~~Liquidity~~
2. ~~ETF~~
3. `TODO` Stock
4. `TODO` Bond
5. ~~Esci dal programma~~
- `todo` Rimettere limite liquidità negativa
- `todo` Controlli più strict sui broker (adesso se sbagli input defaulta a 0 fee)
- `TODO` Gestione separata intermediari 
- `TODO` Conversione valute

In [ ]:
def main_menu():
    print("\n1. Liquidità")
    print("2. ETF")
    print("3. Azioni")
    print("4. Obbligazioni")
    print("5. Esci dal programma")

while True:
    main_menu()
    choice = input("Seleziona operazione (1 2 3 4 5): ")

    # CASH
    if choice == '1':

        date = get_date(df)

        brk = int(input("Seleziona intermediario (SIM) (1. Fineco, 2. BG Saxo): "))
        brokers = {
            1: "Fineco",
            2: "BG Saxo",
        }
        broker = brokers.get(brk, "SIM non riconosciuto")
        
        cash = float(input("Inserisci contante da depositare o prelevare: "))
        if cash == 0.0:
            raise ValueError("Il contante inserito non può essere 0€.\nPositivo se depositato, negativo se prelevato.")
        
        
        df = newrow_cash(df, date, cash, broker)
    
    # ETF
    elif choice == '2':

        date = get_date(df)

        brk = int(input("Seleziona intermediario (SIM) (1. Fineco, 2. BG Saxo): "))
        isin = input("Inserisci ISIN: ")

        qt = input("Inserisci quantità (intero positivo): ")
        if not (qt.isdigit() and int(qt) > 0):
            raise ValueError("Quantità non valida. Deve essere un intero positivo.")
        else:
            quantity = int(qt)


        price = float(input("Inserisci prezzo unitario (negativo se acquisto): "))
        if price == 0:
            raise ValueError("Il prezzo non può essere 0€.\nNegativo se acquisto, positivo se vendita.")
        
        broker, fee = broker_fee(brk, "ETF") 

        if price < 0:   # buy
            buy = True
            difference = df["Liquidita Attuale"].iloc[-1] + (quantity * price - fee)
            #if difference < 0:
            #    raise ValueError(f"Insufficient funds: {df["Liquidita Attuale"].iloc[-1]}€ {quantity * price - fee}€ = {difference}€")
        
        else:           # sell
            buy = False
        
        ter, err = fetch_ter(isin)
        if not ter:
            print(err)
            ter = input("Inserisci TER manualmente: ")

        df = newrow_etf(df, date, isin, quantity, price, ter, broker, fee, buy)

    
    elif choice == '3':
        print("Implementa Azioni")

    elif choice == '4':
        print("Implementa Obbligazioni")
        
    elif choice == '5':
        print("Esco dal programma...")
        break
    
    else:
        print("Scelta non valida. Riprova.")



1. Liquidità
2. ETF
3. Azioni
4. Obbligazioni
5. Esci dal programma



1. Liquidità
2. ETF
3. Azioni
4. Obbligazioni
5. Esci dal programma
Invalid choice! Please try again.

1. Liquidità
2. ETF
3. Azioni
4. Obbligazioni
5. Esci dal programma
Goodbye!


## Salva il report
- Di default sovrascrive `report.csv`
- Per salvare invece in nuovo file, cambiare `report.csv` nel nome desiderato

In [ ]:
df.to_csv("reports/report.csv", index=False)    # Esempio nuovo file: reports/nuovo-report.csv